In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

In [28]:
# Configure the Safari web driver
driver = webdriver.Safari()

# Navigate to the search results page
url = 'https://gov.texas.gov/news/archive'
driver.get(url)

# Wait for the page to finish loading
wait = WebDriverWait(driver, 20)

# Extract the links from the page
soup = BeautifulSoup(driver.page_source, 'html.parser')
response = soup.find(class_="l-content columns small-12 medium-7 large-8")
links = response.find_all('a')

# for link in links[1:10]:
#     print(link)
#     print()
    
driver.quit()    

<a class="readMore" href="https://gov.texas.gov/news/post/governor-abbott-tea-release-teacher-vacancy-recommendations">Governor Abbott, TEA Release Teacher Vacancy Recommendations</a>

<a class="readMore" href="https://gov.texas.gov/news/post/governor-abbott-txdot-announce-record-100-billion-10-year-transportation-plan">Governor Abbott, TxDOT Announce Record $100 Billion 10-Year Transportation Plan</a>

<a class="readMore" href="https://gov.texas.gov/news/post/governor-abbott-announces-film-friendly-texas-designation-for-the-city-of-cameron">Governor Abbott Announces Film Friendly Texas Designation For The City Of Cameron</a>

<a class="readMore" href="https://gov.texas.gov/news/post/governor-abbott-launches-statewide-street-takeover-task-force">Governor Abbott Launches Statewide Street Takeover Task Force</a>

<a class="readMore" href="https://gov.texas.gov/news/post/governor-abbott-outlines-priorities-for-texas-thriving-economy">Governor Abbott Outlines Priorities For Texas’ Thriving

In [29]:
def get_text(text_list):
    arr = []
    for t in text_list:
        arr.append(t.get_text())
    return " ".join(arr)

In [45]:
driver = webdriver.Safari()

ols_map = {}
no_ols = {}
for link in links:
    href = link.get('href')
    if href.startswith('http'):
        # Navigate to the page
        driver.get(href)

        # Extract the relevant text
        page_soup = BeautifulSoup(driver.page_source, 'html.parser')
        texts = page_soup.find_all('p')
        text = get_text(texts).lower()
        
        # Store the text in a map
        if "operation lone star" in text:
            ols_map[link.get_text().lower()] = text
        else:
            no_ols[link.get_text().lower()] = text

driver.quit()

In [47]:
# results_map[links[0].get_text()]
print(len(no_ols))

62


In [48]:
print(results_map["Governor Abbott, TEA Release Teacher Vacancy Recommendations"])

February 24, 2023 | Austin, Texas 
      |
        
          Press Release
 Governor Greg Abbott today announced the release of the Final Report of the Teacher Vacancy Task Force that focuses on three primary areas of policy recommendations to address school district staffing challenges in Texas. In March 2022, Governor Abbott directed Texas Education Agency (TEA) Commissioner Mike Morath to immediately create a task force to help school districts address ongoing staffing retention and recruitment challenges. "Educating Texas students is crucial for the continued success of our great state, and teachers play a pivotal role in that success," said Governor Abbott. "I thank the teachers and school leaders on this task force for their work addressing full-time and substitute teacher vacancies in Texas schools. Their recommendations will help ensure that best practices and resources are available for teacher recruitment and retention. Working with the Texas legislature, we will develop and

## Scraping news for multiple dates

March 2021- March 2022 (to present?)

In [107]:
import json

years = ['2022', '2023'] #'2021',
months = ['01', '02', '03', '04','05', '06','07', '08', '09', '10','11', '12']
driver = webdriver.Safari()

# results_map = {}
# no_results_map = {}
for year in years:
    if year == '2023':
        y_months = months[0:2]
    else:
        y_months = months
        
    year_map = {}
    no_year_map = {}
    for month in y_months:
        month_map = {}
        no_month_map = {}
        
        url = f'https://gov.texas.gov/news/archive/{year}/{month}'
        driver.get(url)
        print(url)
        
        # Wait for the page to finish loading
        wait = WebDriverWait(driver, 100)
        
        # Extract the links from the page
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        # print(soup.prettify())
        nsoup = soup.find(class_="l-siteContent")
        response = nsoup.find(class_="l-content columns small-12 medium-7 large-8")
        links = response.find_all('a')

        for link in links:
            href = link.get('href')
            if href.startswith('http'):
                # Navigate to the page
                driver.get(href)

                # Extract the relevant text
                page_soup = BeautifulSoup(driver.page_source, 'html.parser')
                texts = page_soup.find_all('p')
                text = get_text(texts).lower()

                # Store the text in a map
                if "operation lone star" in text:
                    month_map[link.get_text().lower()] = text
                else:
                    no_month_map[link.get_text().lower()] = text
        
        with open(f"archive/OLS/{year}/{month}.json", "w") as outfile:
            # write the dictionary to the file as JSON
            json.dump(month_map, outfile)
        
        with open(f"archive/noOLS/{year}/{month}.json", "w") as outfile:
            # write the dictionary to the file as JSON
            json.dump(no_month_map, outfile)
            
        year_map[month] = month_map
        no_year_map[month] = no_month_map
    
    results_map[year] = year_map
    no_results_map[year] = no_year_map
            
driver.quit()    

https://gov.texas.gov/news/archive/2022/01
https://gov.texas.gov/news/archive/2022/02
https://gov.texas.gov/news/archive/2022/03
https://gov.texas.gov/news/archive/2022/04
https://gov.texas.gov/news/archive/2022/05
https://gov.texas.gov/news/archive/2022/06
https://gov.texas.gov/news/archive/2022/07
https://gov.texas.gov/news/archive/2022/08
https://gov.texas.gov/news/archive/2022/09
https://gov.texas.gov/news/archive/2022/10
https://gov.texas.gov/news/archive/2022/11
https://gov.texas.gov/news/archive/2022/12
https://gov.texas.gov/news/archive/2023/01
https://gov.texas.gov/news/archive/2023/02


In [135]:
total = 0
for year in results_map:
    for month in results_map[year]:
        total = total + len(results_map[year][month])
print(total, "total articles containing \"Operation Lone Star\"")

for year in no_results_map:
    for month in no_results_map[year]:
        total = total + len(no_results_map[year][month])
print(total, "total articles not containing \"Operation Lone Star\"")


93 total articles containing "Operation Lone Star"
1549 total articles not containing "Operation Lone Star"


## Word Frequency

In [133]:
import nltk
from nltk import FreqDist

lets work at word frequency per month

In [228]:
import numpy as np

years = ['2021', '2022', '2023']
months = ['01', '02', '03', '04','05', '06','07', '08', '09', '10','11', '12']

all_results_dict = {}
for year in years:
    if year == '2023':
            months = ['01', '02']
    for month in months:
        # print(results_map[year][month])
        text_month = " ".join(results_map[year][month].values())
        all_results_dict[f'{month}/{year}'] = text_month
        
# print(len(all_results_dict))
# print(all_results_dict.items())

dtype = np.dtype([
    ('id', int),
    ('date', object),
    ('text', object),
])

data_list = [(i, k, "".join(list(v))) for i, (k, v) in enumerate(all_results_dict.items())]

all_results_np = np.array(data_list, dtype=dtype)

print(all_results_np[4])

(4, '05/2021', 'may 27, 2021 | austin, texas \n      |\n        \n          press release\n governor greg abbott today held a press conference in fort worth where he highlighted the state\'s efforts to secure the border and stop the trafficking of fentanyl into texas. in response to an unprecedented surge in fentanyl coming across the border, the governor announced his intention to sign senate bill 768 (huffman/johnson) that will create a specific criminal offense, beginning with a third degree felony, for the manufacturing or delivery of fentanyl. the governor pointed to president biden\'s open border policies as a cause behind the surge in fentanyl, and urged the president\xa0to address this crisis and secure the border. governor abbott called upon parents to educate their children about the potentially deadly consequences of taking fentanyl, or unknowingly taking other drugs laced with fentanyl.\xa0 "lives are at stake as dangerous drugs like fentanyl pour into our communities at an

In [232]:
all_results_np['date']

array(['01/2021', '02/2021', '03/2021', '04/2021', '05/2021', '06/2021',
       '07/2021', '08/2021', '09/2021', '10/2021', '11/2021', '12/2021',
       '01/2022', '02/2022', '03/2022', '04/2022', '05/2022', '06/2022',
       '07/2022', '08/2022', '09/2022', '10/2022', '11/2022', '12/2022',
       '01/2023', '02/2023'], dtype=object)

In [237]:
# all_text = []

# for year in results_map.values():
#     year_text = []
#     for month in year.values():
#         text_month = " ".join(month.values())
#         year_text.append(text_month)
#     all_text.append(year_text)
    
    
freqs = {}

for _,d,t in all_results_np:
    words = nltk.word_tokenize(t)
    fdist = FreqDist(words)
    freqs[d] = fdist
    
for k, v in freqs.items():
    print(k, ':')
    for word, frequency in v.most_common(10):
        print(f"{word}: {frequency}")
    
# test_month = results_map['2021']['03']
# text_month = " ".join(test_month.values())

# words = nltk.word_tokenize(text_month)

# fdist = FreqDist(words)

# for word, frequency in fdist.most_common(10):
#     print(f"{word}: {frequency}")

01/2021 :
02/2021 :
03/2021 :
the: 119
to: 76
,: 74
and: 59
.: 51
border: 45
texas: 31
governor: 30
of: 27
in: 22
04/2021 :
the: 59
,: 38
and: 33
to: 28
.: 25
of: 22
border: 17
governor: 14
texas: 11
a: 8
05/2021 :
the: 26
,: 25
to: 22
.: 16
fentanyl: 14
in: 11
and: 11
of: 11
a: 9
governor: 8
06/2021 :
and: 12
,: 10
border: 7
to: 7
the: 7
.: 6
texas: 4
over: 4
vice: 3
president: 3
07/2021 :
08/2021 :
the: 88
,: 80
.: 59
to: 53
of: 45
and: 45
border: 44
in: 30
texas: 27
a: 27
09/2021 :
the: 194
,: 119
and: 95
to: 93
.: 85
of: 83
border: 55
texas: 46
in: 42
governor: 41
10/2021 :
the: 79
,: 63
to: 48
.: 42
and: 38
border: 35
governor: 32
of: 31
texas: 26
in: 25
11/2021 :
the: 26
,: 20
and: 20
to: 20
.: 17
texas: 11
of: 10
mexican: 9
in: 8
governor: 7
12/2021 :
the: 29
,: 15
to: 13
and: 12
governor: 11
.: 11
border: 11
in: 10
of: 10
texas: 7
01/2022 :
02/2022 :
the: 86
,: 76
.: 52
to: 44
and: 39
texas: 34
in: 33
of: 30
governor: 22
's: 22
03/2022 :
the: 240
,: 212
.: 131
and: 120
to: 96
o

## Without Driver

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
# Make request to the search results page
url = "https://gov.texas.gov/news/archive"
response = requests.get(url)

# Parse HTML using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Find all links in search results
# results = soup.find_all("a")

job_elements = soup.find_all("section", class_="l-content columns small-12 medium-7 large-8 ")

In [5]:
job_elements

[]